<a href="https://colab.research.google.com/github/gacerioni/Redis-VectorDB-LLM-RAG-Workshop/blob/main/01-RedisJSON_Search/01-RedisJSON_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RedisJSON e RediSearch

![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)

Esse notebook é uma adaptação simplificada do material que já existe na documentação. Aqui, é o nosso viés para Demo, mas o conhecimento é equivalente.

Para uma experiência premium, como a que eu quero que vocês tenham, recomendo MUITO utilizar o Redis Insight (App ou Web) pra apoiar na visualização dos dados.

https://redis.com/redis-enterprise/redis-insight/

## Setup Rápido - e testes pra ver se tá tudo redondo antes de iniciar o lab

In [44]:
# Install the requirements
!pip install -q redis

In [45]:
%%sh
curl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg
echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list
sudo apt-get update  > /dev/null 2>&1
sudo apt-get install redis-stack-server  > /dev/null 2>&1
redis-stack-server --daemonize yes


deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb jammy main
Starting redis-stack-server, database path /var/lib/redis-stack


gpg: cannot open '/dev/tty': No such device or address
curl: (23) Failed writing body


In [46]:
import os

REDIS_HOST = os.getenv("REDIS_HOST", "localhost")
REDIS_PORT = os.getenv("REDIS_PORT", "6379")
REDIS_PASSWORD = os.getenv("REDIS_PASSWORD", "")

# Troque estes valores, caso voce esteja usando o Redis Cloud. Eu estou, rsrs.
REDIS_HOST="redis-19581.c308.sa-east-1-1.ec2.redns.redis-cloud.com"
REDIS_PORT=19581
REDIS_PASSWORD="nhtuquVSLbh2kUt2I86z5QwGu3KrcaYx"

# If SSL is enabled on the endpoint add --tls
if REDIS_PASSWORD!="":
  os.environ["REDIS_CONN"]=f"-h {REDIS_HOST} -p {REDIS_PORT} -a {REDIS_PASSWORD} --no-auth-warning"
else:
  os.environ["REDIS_CONN"]=f"-h {REDIS_HOST} -p {REDIS_PORT}"

# If SSL is enabled on the endpoint, use rediss:// as the URL prefix
REDIS_URL = f"redis://:{REDIS_PASSWORD}@{REDIS_HOST}:{REDIS_PORT}"
INDEX_NAME = f"qna:idx"

# Test Redis connection
!redis-cli $REDIS_CONN PING

PONG


In [47]:
import redis
r = redis.Redis(
  host=REDIS_HOST,
  port=REDIS_PORT,
  password=REDIS_PASSWORD)
r.ping()

True

## RedisJSON

**RedisJSON** adiciona o tipo de dado **JSON** ao Redis para que você possa trabalhar com dados JSON **nativamente** no Redis, sem tratar o JSON inteiro como uma grande string e constantemente serializar/desserializar JSON no cliente.

Isso também significa que eu estou criando um **modelo/schema**. Os campos do JSON são tipados e possuem metacaracterísticas que vão ajudar muito vocês na hora de buscar dados no Redis. Buscas multi-facetadas, de vetor, de geolocalização, com filtros e transoformações que ocorrerão do lado do Redis, evitando multiplos round trips para o backend.

*Em outras palavras, você tira o que quer do Redis indo só uma vez até ele, inclusive filtrando exatamente o que você quer que passe pelo fio até sua aplicação.*


-----


Com uma biblioteca cliente como Python, você pode usar comandos como `redis.json().get()` e `redis.json().set()`, e no Redis CLI, JSON.GET, JSON.SET entre outros.

Eu, particularmente, prefiro usar as bibliotecas de Object Mapping, como a [REDISOM](https://redis.io/docs/latest/integrate/redisom-for-python/).

-----

Veja a lista completa de comandos do RedisJSON aqui: https://redis.io/commands/?group=json

Documentação Python: https://redis-py.readthedocs.io/en/stable/redismodules.html#redisjson-commands


In [48]:
schools = [
    {
        "name": "Colégio Bandeirantes",
        "description": "Abrangendo 10 estados, o currículo premiado desta escola inclui um sistema de leitura abrangente (do reconhecimento de letras e fonética à leitura de livros completos), além de matemática, ciências, estudos sociais e até filosofia.",
        "class": "independente",
        "type": ["tradicional"],
        "address": {"city": "São Paulo", "street": "Rua Estela"},
        "students": 342,
        "location": {"latitude": -23.550520, "longitude": -46.633308},
        "status_log": ["novo", "em operação"],
        "teachers": [
            {
                "name": "Ana Silva",
                "subjects": ["Matemática", "Física"]
            },
            {
                "name": "Carlos Souza",
                "subjects": ["História", "Geografia"]
            }
        ],
        "tags": "excelência acadêmica, tecnologia, inovação"
    },
    {
        "name": "Escola Maria Imaculada (Chapel School)",
        "description": "A Garden School é uma nova e inovadora experiência de ensino e aprendizado ao ar livre, oferecendo atividades ricas e variadas em um ambiente natural para crianças e famílias.",
        "class": "estadual",
        "type": ["floresta", "montessori"],
        "address": {"city": "São Paulo", "street": "Avenida Vicente Rao"},
        "students": 1452,
        "location": {"latitude": -23.621993, "longitude": -46.699687},
        "status_log": ["novo", "em operação"],
        "teachers": [
            {
                "name": "Mariana Costa",
                "subjects": ["Biologia", "Química"]
            },
            {
                "name": "Rafael Lima",
                "subjects": ["Português", "Literatura"]
            }
        ],
        "tags": "educação ao ar livre, inovação, sustentabilidade"
    },
    {
        "name": "Colégio São Luís",
        "description": "A Gillford School é um centro de aprendizado inclusivo que acolhe pessoas de todos os estilos de vida, convidando-as a assumir seu papel como agentes regenerativos, criando novos caminhos para o futuro e incitando um movimento internacional de transformação cultural, territorial e social.",
        "class": "privada",
        "type": ["democrática", "waldorf"],
        "address": {"city": "São Paulo", "street": "Rua Haddock Lobo"},
        "students": 721,
        "location": {"latitude": -23.558704, "longitude": -46.660213},
        "status_log": ["novo", "em operação", "fechada"],
        "teachers": [
            {
                "name": "Beatriz Mendes",
                "subjects": ["Filosofia", "Sociologia"]
            },
            {
                "name": "Fernando Almeida",
                "subjects": ["Artes", "Educação Física"]
            }
        ],
        "tags": "inclusão, transformação social, interdisciplinaridade"
    },
    {
        "name": "Escola Sesc de Ensino Médio",
        "description": "A filosofia por trás da Forest School baseia-se no desejo de proporcionar às crianças pequenas uma educação que incentive a apreciação do mundo amplo na natureza, enquanto alcançam independência, confiança e alta autoestima.",
        "class": "independente",
        "type": ["floresta", "montessori", "democrática"],
        "address": {"city": "Rio de Janeiro", "street": "Rua Jacarepaguá"},
        "students": 1200,
        "location": {"latitude": -22.972250, "longitude": -43.375162},
        "status_log": ["novo", "em operação"],
        "teachers": [
            {
                "name": "Luciana Oliveira",
                "subjects": ["Ciências", "Matemática"]
            },
            {
                "name": "Paulo Santos",
                "subjects": ["História", "Geografia"]
            }
        ],
        "tags": "natureza, independência, confiança"
    },
    {
        "name": "Escola Móbile",
        "description": "A Escola Móbile oferece um programa acadêmico rigoroso combinado com foco em pensamento crítico e criatividade, preparando os alunos para o ensino superior e além.",
        "class": "privada",
        "type": ["tradicional"],
        "address": {"city": "São Paulo", "street": "Rua Cotovia"},
        "students": 600,
        "location": {"latitude": -23.592920, "longitude": -46.676201},
        "status_log": ["novo", "em operação"],
        "teachers": [
            {
                "name": "Juliana Pereira",
                "subjects": ["Química", "Biologia"]
            },
            {
                "name": "Ricardo Fernandes",
                "subjects": ["Português", "Inglês"]
            }
        ],
        "tags": "rigor acadêmico, pensamento crítico, criatividade"
    },
    {
        "name": "Colégio Santa Cruz",
        "description": "O Colégio Santa Cruz é conhecido por seu forte desempenho acadêmico e compromisso com a responsabilidade social, promovendo um senso de comunidade entre os alunos.",
        "class": "privada",
        "type": ["tradicional", "religiosa"],
        "address": {"city": "São Paulo", "street": "Rua Orobó"},
        "students": 1300,
        "location": {"latitude": -23.547680, "longitude": -46.712750},
        "status_log": ["novo", "em operação"],
        "teachers": [
            {
                "name": "Gabriela Rocha",
                "subjects": ["Religião", "Filosofia"]
            },
            {
                "name": "Eduardo Matos",
                "subjects": ["Educação Física", "Ciências"]
            }
        ],
        "tags": "desempenho acadêmico, responsabilidade social, comunidade"
    },
    {
        "name": "Colégio Rio Branco",
        "description": "O Colégio Rio Branco oferece um currículo diversificado com forte ênfase em línguas, artes e ciências, preparando os alunos para a cidadania global.",
        "class": "privada",
        "type": ["internacional"],
        "address": {"city": "São Paulo", "street": "Rua Alves Guimarães"},
        "students": 1100,
        "location": {"latitude": -23.555451, "longitude": -46.670912},
        "status_log": ["novo", "em operação"],
        "teachers": [
            {
                "name": "Marcelo Andrade",
                "subjects": ["Inglês", "Espanhol"]
            },
            {
                "name": "Sofia Ribeiro",
                "subjects": ["Artes", "História"]
            }
        ],
        "tags": "línguas, artes, cidadania global"
    }
]



# carregar os dados do Redis... algumas escolas do Brasilzão
for id, school in enumerate(schools):
    #print(school)
    r.json().set(f"school_json:{id}", '.', school)

In [49]:
!redis-cli $REDIS_CONN JSON.GET school_json:1 $
!redis-cli $REDIS_CONN keys 'school_json:*'

"[{\"name\":\"Escola Maria Imaculada (Chapel School)\",\"description\":\"A Garden School \xc3\xa9 uma nova e inovadora experi\xc3\xaancia de ensino e aprendizado ao ar livre, oferecendo atividades ricas e variadas em um ambiente natural para crian\xc3\xa7as e fam\xc3\xadlias.\",\"class\":\"estadual\",\"type\":[\"floresta\",\"montessori\"],\"address\":{\"city\":\"S\xc3\xa3o Paulo\",\"street\":\"Avenida Vicente Rao\"},\"students\":1452,\"location\":{\"latitude\":-23.621993,\"longitude\":-46.699687},\"status_log\":[\"novo\",\"em opera\xc3\xa7\xc3\xa3o\"],\"teachers\":[{\"name\":\"Mariana Costa\",\"subjects\":[\"Biologia\",\"Qu\xc3\xadmica\"]},{\"name\":\"Rafael Lima\",\"subjects\":[\"Portugu\xc3\xaas\",\"Literatura\"]}],\"tags\":\"educa\xc3\xa7\xc3\xa3o ao ar livre, inova\xc3\xa7\xc3\xa3o, sustentabilidade\"}]"
1) "school_json:4"
2) "school_json:6"
3) "school_json:1"
4) "school_json:5"
5) "school_json:3"
6) "school_json:0"
7) "school_json:2"


In [50]:
# Pegue um JSON direto, desde a raiz com $
res=r.json().get('school_json:0','$')
print(res)


[{'name': 'Colégio Bandeirantes', 'description': 'Abrangendo 10 estados, o currículo premiado desta escola inclui um sistema de leitura abrangente (do reconhecimento de letras e fonética à leitura de livros completos), além de matemática, ciências, estudos sociais e até filosofia.', 'class': 'independente', 'type': ['tradicional'], 'address': {'city': 'São Paulo', 'street': 'Rua Estela'}, 'students': 342, 'location': {'latitude': -23.55052, 'longitude': -46.633308}, 'status_log': ['novo', 'em operação'], 'teachers': [{'name': 'Ana Silva', 'subjects': ['Matemática', 'Física']}, {'name': 'Carlos Souza', 'subjects': ['História', 'Geografia']}], 'tags': 'excelência acadêmica, tecnologia, inovação'}]


In [51]:
# Retorne apenas a propriedade nome
res=r.json().get('school_json:0','$.name')
print(res)

# Embedded object ($.address) - Um Address pertence a uma Escola - 1:1
res=r.json().get('school_json:0','$.address')
print(res)

# E agora um elemento de um array
res=r.json().get('school_json:0','$.status_log[0]')
print(res)

['Colégio Bandeirantes']
[{'city': 'São Paulo', 'street': 'Rua Estela'}]
['novo']


In [52]:
# Ler a quantidade de students de uma escola X. Neste caso, a primeira!
students=r.json().get('school_json:0','$.students')
print(students)

# Mudar pra 350 alunos e refazer o mesmo comando.
# sempre atomico, sempre O(1).
r.json().set('school_json:0','$.students',350)
r.json().get('school_json:0','$.students')

[342]


[350]

In [53]:
# Incremento atomico do numero de estudandes da escola-alvo... sempre O(1)
# Pode clicar algumas vezes! O numincrby vai retornar o novo numero de Estudantes da Escola, assim como na CLI/API.
r.json().numincrby('school_json:0','$.students',1)

[351]

## RediSearch - Dando mais poderes para o que já é brabo

**RediSearch** adiciona a capacidade de consultar dados nas suas estruturas de dados **HASH** ou **JSON**, **essencialmente transformando o Redis em um banco de dados de documentos.** Sim.

Com o RediSearch, você declara índices uma vez e, em seguida, cada objeto de banco de dados que corresponder ao prefixo definido no índice será automaticamente e em tempo real adicionado ao índice.

Naturalmente, o que já estiver no Redis será indexado também, a não ser que você não queira. Existem vários designs diferentes para esse tema, incluindo indices pequenos de com curto lifespan. Alguns clientes usam esse modelo para tornar a experiencia de um cliente mais fluida e com menor percepção de latência.\
*Por exemplo, você cria um determinado indice que só vai servir durante a jornada de um cliente logado, até o checkout, e depois vc manda o índice pro beleléu.*


-----

Para a lista completa de comandos do RediSearch, veja: https://redis.io/commands/?group=search

Documentação Python: https://redis-py.readthedocs.io/en/stable/redismodules.html#redisearch-commands

In [56]:
# Aqui eu quero trazer a sua atencao para o schema.
# Notem como eu explico para o Redis como tratar o JSON como um objeto mesmo, considerando seus atributos/propriedades.
# estamos apenas comecando! esse indice nem inclui geoloc ou vetor (ainda, rsrs, vamos ver juntos em breve)

from redis.commands.search.field import (
    NumericField,
    TagField,
    TextField,
    VectorField,
)
from redis.commands.search.indexDefinition import IndexDefinition, IndexType
from redis.commands.search.query import Query

# Excluir o índice se ele já existir
try:
    r.ft("idx:schools_json").dropindex(delete_documents=False)
except:
    pass

# E aqui criamos o index pra valer. Nem vou colocar vector e geoloc agora, pra gente nao confundir.
schema = (
    TextField("$.name", as_name="name"),
    TextField("$.description", as_name="description"),
    TagField("$.address.city", as_name="city"),
    NumericField("$.students", as_name="students"),
    TagField("$.tags", as_name="tags", separator=",")
    )
r.ft("idx:schools_json").create_index(schema,
                    definition=IndexDefinition(prefix=["school_json:"],
                    index_type=IndexType.JSON)
                    )

b'OK'

### Testando o Setup com uma Full-Text search

Também existem tags para exact match, belê?


Não quero te confundir agora, mas os casos de uso vão do básico até avançadíssimos, como você pode ver [aqui](https://redis.io/docs/latest/develop/interact/search-and-query/advanced-concepts/).

In [57]:
import pandas as pd
# Retorne o Documento inteiro! Aqui fizemos uma Full-Text Search por nature.
res=r.ft("idx:schools_json").search("nature")
res_df = pd.DataFrame([t.__dict__ for t in res.docs ])
res_df

,id,payload,json
0,school_json:1,None,"{""name"":""Escola Maria Imaculada (Chapel School..."


In [58]:
# Retorne apenas alguns campos, para evitar passar coisas e metadados desnecessários pelo fio, do Redis para sua App.
query=Query("nature") \
   .return_field("$.address.city", as_field="city") \
   .return_field("$.name", as_field="name")
res=r.ft("idx:schools_json").search(query)
res_df = pd.DataFrame([t.__dict__ for t in res.docs ])
res_df

,id,payload,city,name
0,school_json:1,None,São Paulo,Escola Maria Imaculada (Chapel School)


In [59]:
# Busca Multi-Field, um pouquinho mais complexa
# Notem o uso de {} para exact match de tags
# Notem tambem que estamos vendo que o Redis eh binary-safe...
# A Cidade, sendo uma tag, casa certinho, considerando o 'ã' de São Paulo
query=Query('@city:{São Paulo} @students:[0, 10000]') \
   .return_field("$.address.city", as_field="city") \
   .return_field("$.name", as_field="name") \
   .return_field("$.students", as_field="students")
res=r.ft("idx:schools_json").search(query)
#print(res)
res_df = pd.DataFrame([t.__dict__ for t in res.docs ])
res_df


,id,payload,city,name,students
0,school_json:5,None,São Paulo,Colégio Santa Cruz,1300
1,school_json:4,None,São Paulo,Escola Móbile,600
2,school_json:6,None,São Paulo,Colégio Rio Branco,1100
3,school_json:0,None,São Paulo,Colégio Bandeirantes,351
4,school_json:2,None,São Paulo,Colégio São Luís,721
5,school_json:1,None,São Paulo,Escola Maria Imaculada (Chapel School),1452


# Casos de uso poderosos e mais avançados
(não são complexos, é tranquilo entender)

### Busca por Tags - Encontrando escolas com características específicas

Nesta seção, vamos realizar buscas utilizando tags. Tags permitem buscas exatas em campos específicos.

In [60]:
# Busca por tags exatas - binary safe - usando o melhor idioma do PLANETA: PT-BR
query = Query('@tags:{inovação}') \
    .return_field("$.address.city", as_field="city") \
    .return_field("$.name", as_field="name")
res = r.ft("idx:schools_json").search(query)
res_df = pd.DataFrame([t.__dict__ for t in res.docs])
print(res_df)

              id payload       city                                    name
0  school_json:0    None  São Paulo                    Colégio Bandeirantes
1  school_json:1    None  São Paulo  Escola Maria Imaculada (Chapel School)


### Busca por Intervalo - Quantos alunos estão matriculados?

Busca por intervalo é o que você vai encontrar na doc oficial como "Range queries"

Para realizar buscas por intervalos em campos numéricos, como a quantidade de alunos em uma escola, podemos utilizar essas consultas de intervalo.

**Não se assustem com `inf+` e `inf-`!**\
Esses termos são utilizados para representar o intervalo máximo e mínimo, respectivamente. inf+ significa infinito positivo (o valor mais alto possível) e inf- significa infinito negativo (o valor mais baixo possível). Eles são úteis quando você quer incluir todos os valores acima ou abaixo de um determinado limite.

In [72]:
# Aqui, vamos realizar buscas baseadas em intervalos numéricos. Neste caso, a quantidade de alunos.
# TODO - Colocar mais ranges e mudar as cidades

# Busca por escolas com entre 300 e 1000 estudantes (inclusivo)
query = Query('@students:[300 1000]') \
    .return_field("$.address.city", as_field="city") \
    .return_field("$.name", as_field="name") \
    .return_field("$.students", as_field="students")
res = r.ft("idx:schools_json").search(query)
res_df = pd.DataFrame([t.__dict__ for t in res.docs])
print("--------------------------------------------------------------------")
print("Escolas que têm entre 300 e 1000 estudantes (inclusive)")
print(res_df)
print("--------------------------------------------------------------------")

# Busca por escolas com mais de 1000 alunos (exclusivo)
query = Query('@students:(1000 inf+]') \
    .return_field("$.address.city", as_field="city") \
    .return_field("$.name", as_field="name") \
    .return_field("$.students", as_field="students")

res_df = pd.DataFrame([t.__dict__ for t in res.docs])
print("--------------------------------------------------------------------")
print("Escolas que têm mais de 1000 estudantes (exclusive)")
print(res_df)
print("--------------------------------------------------------------------")


# Busca por escolas com menos de 300 alunos (inclusivo)
query = Query('@students:[inf- 300]') \
    .return_field("$.address.city", as_field="city") \
    .return_field("$.name", as_field="name") \
    .return_field("$.students", as_field="students")

res_df = pd.DataFrame([t.__dict__ for t in res.docs])
print("--------------------------------------------------------------------")
print("Escolas que têm menos de 300 estudantes (inclusive)")
print(res_df)
print("--------------------------------------------------------------------")


# Busca por escolas com menos de 300 alunos (exclusivo)
query = Query('@students:[inf- 300)') \
    .return_field("$.address.city", as_field="city") \
    .return_field("$.name", as_field="name") \
    .return_field("$.students", as_field="students")

res_df = pd.DataFrame([t.__dict__ for t in res.docs])
print("--------------------------------------------------------------------")
print("Escolas que têm menos de 300 estudantes (exclusive)")
print(res_df)
print("--------------------------------------------------------------------")



--------------------------------------------------------------------
Escolas que têm entre 300 e 1000 estudantes (inclusive)
              id payload       city                  name students
0  school_json:4    None  São Paulo         Escola Móbile      600
1  school_json:0    None  São Paulo  Colégio Bandeirantes      351
2  school_json:2    None  São Paulo      Colégio São Luís      721
--------------------------------------------------------------------
--------------------------------------------------------------------
Escolas que têm mais de 1000 estudantes (exclusive)
              id payload       city                  name students
0  school_json:4    None  São Paulo         Escola Móbile      600
1  school_json:0    None  São Paulo  Colégio Bandeirantes      351
2  school_json:2    None  São Paulo      Colégio São Luís      721
--------------------------------------------------------------------
--------------------------------------------------------------------
Escolas que

### Fuzzy Match - "Serasse" eu sei escrever essa palavra?

Pra fechar, apenas um exemplo leve de comunicação realista: podemos estar buscando sem saber escrever direito o termo. Esse é apenas um das dezenas de exemplos que vocês vão ter acesso na doc do Redis. Tem de redução de radicais, levando em consideração o contexto e cultura de uma determinada Língua, etc.

Aqui, vamos apenas fazer uma full-text search procurando por alguma coisa meio esquisita, e vamos ver se o Redis consegue ajudar.

In [61]:
# Fuzzy matching!
# %algumacoisa% significa todos os termos com distância de Levenshtein de 1 em relação a ele.
# Use múltiplos pares de % para aumentar a distância de Levenshtein.

query=Query('%%Bondeirants%%') \
   .return_field("$.address.city", as_field="city") \
   .return_field("$.name", as_field="name") \
   .return_field("$.students", as_field="students")
res=r.ft("idx:schools_json").search(query)
#print(res)
res_df = pd.DataFrame([t.__dict__ for t in res.docs ])
res_df

,id,payload,city,name,students
0,school_json:0,None,São Paulo,Colégio Bandeirantes,351
